# Spatial Statistics with Voyager

Based on the following tutorials:
* https://pachterlab.github.io/voyager/articles/visium_10x.html
* https://pachterlab.github.io/voyager/articles/vig1_visium_basic.html
* https://pachterlab.github.io/voyager/articles/vig2_visium.html
* https://pachterlab.github.io/voyager/articles/visium_10x_spatial.html

In [ ]:
library(dplyr)
library(Voyager)
library(SpatialExperiment)
library(SpatialFeatureExperiment)
library(SingleCellExperiment)
library(ggplot2)
library(scater)
library(rlang)
library(scran)
library(scuttle)
library(terra)
library(sf)
library(rmapshaper)
library(scran)
library(stringr)
library(EBImage)
library(patchwork)
library(bluster)
library(rjson)
theme_set(theme_bw())

In [ ]:
# Layout
custom_theme <- function() {
  theme_bw() +
    theme(
      legend.text = element_text(size = 14),
      legend.title = element_text(size = 16, face = "bold"),
      axis.text = element_text(size = 12),
      axis.title = element_text(size = 14, face = "bold"),
      legend.position = "right",
      legend.box.just = "right"
    )
}
options(repr.plot.width = 20, repr.plot.height = 16)

In [ ]:
data_dir <- R.utils::getAbsolutePath('../../data')
mouse_dir <- glue::glue("{data_dir}/Visium_Mouse_Olfactory_Bulb/outs")

## Visium Files

### Scale Factors

The scalefactors_json.json file contains image metadata:
* **tissue_hires_scalef** and **tissue_lowres_scalef** are the ratio of the size of the high resolution (but not full resolution) and low resolution H&E image to the full resolution image.
* **fiducial_diameter_fullres** is the diameter of each fiducial spot used to align the spots to the H&E image in pixels in the full resolution image.
* **spot_diameter_fullres** is the diameter of each Visium spot in the full resolution H&E image in pixels. 

In [ ]:
fromJSON(file = glue::glue("{mouse_dir}/spatial/scalefactors_json.json"))

### Tissue Metadata

The tissue_positions_list.csv file contains information about each spot/barcode:
* **in_tissue** indicates whether each spot is in tissue (in_tissue, 1 means yes and 0 means no) as automatically detected by 
Space Ranger or manually annotated in the Loupe browser.
* **array_row** and **array_col** are the coordinates on the matrix of spots,
* **pxl_row_in_fullres** and **pxl_col_in_fullres** are the coordinates of the spots in the full resolution 
image.

In [ ]:
head(read.csv(glue::glue("{mouse_dir}/spatial/tissue_positions.csv")))

# Read Visium Data

In [ ]:
#Original way to load Visium Data
#raw_sfe <- SpatialFeatureExperiment::read10xVisiumSFE(dirs = mouse_dir, samples = ".", type = "sparse", data = "raw")
#Voyager::plotImage(raw_sfe)
#transposed_raw_sfe <- SpatialFeatureExperiment::transpose(raw_sfe)

In [ ]:
# Read pre-processed file
raw_sfe <- readRDS(glue::glue("{data_dir}/Visium_Mouse_Olfactory_Bulb.rds"))
Voyager::plotImage(raw_sfe)
transposed_raw_sfe <- raw_sfe

# Perform QC

In [ ]:
is_mt <- str_detect(rowData(transposed_raw_sfe)$symbol, "^mt-")
sum(is_mt)

In [ ]:
qc_sfe <- scuttle::addPerCellQCMetrics(transposed_raw_sfe, subsets = list(mito = is_mt))

In [ ]:
(scater::plotColData(qc_sfe, "sum", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(scater::plotColData(qc_sfe, "detected", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(scater::plotColData(qc_sfe, "subsets_mito_percent", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(patchwork::plot_layout(ncol=3, guides = "collect"))

In [ ]:
scater::plotColData(qc_sfe, x = "sum", y = "subsets_mito_percent", bins = 100) + custom_theme()

In [ ]:
# Normally MITO % is set to 20 - see what effect this has compared to 30.
processed_sfe <- qc_sfe[, qc_sfe$subsets_mito_percent < 20]
processed_sfe <- processed_sfe[rowSums(counts(processed_sfe)) > 0,]

In [ ]:
(plotColData(processed_sfe, "sum", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(plotColData(processed_sfe, "detected", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(plotColData(processed_sfe, "subsets_mito_percent", x = "in_tissue", color_by = "in_tissue") + custom_theme()) +
(plot_layout(ncol=3, guides = "collect"))

In [ ]:
plotColData(processed_sfe, x = "sum", y = "subsets_mito_percent", bins = 100) + custom_theme()

# Before and after QC by Percentage

In [ ]:
(Voyager::plotSpatialFeature(qc_sfe, c("sum"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme() +
Voyager::plotSpatialFeature(qc_sfe, c("detected"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme() +
Voyager::plotSpatialFeature(qc_sfe, c("subsets_mito_percent"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme()) +
(plot_layout(ncol = 2, guides = "collect"))

In [ ]:
colData(processed_sfe)$nCounts <- colSums(counts(processed_sfe))

In [ ]:
(Voyager::plotSpatialFeature(processed_sfe, c("sum"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme() +
Voyager::plotSpatialFeature(processed_sfe, c("detected"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme() +
Voyager::plotSpatialFeature(processed_sfe, c("subsets_mito_percent"), image_id = "lowres", maxcell = 5e4, ncol = 2) + custom_theme()) +
(plot_layout(ncol = 2, guides = "collect"))

# Plotting Metrics in Space

## Counts Per Spot in and out of Tissue

Plot the total unique molecular identifier (UMI) counts per spot.

In [ ]:
violin <- plotColData(processed_sfe, "nCounts", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top") + custom_theme()
spatial <- plotSpatialFeature(processed_sfe, "nCounts", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary", 
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) + custom_theme()
violin + spatial

In [ ]:
colData(processed_sfe)$nGenes <- colSums(counts(processed_sfe) > 0)

In [ ]:
violin <- scater::plotColData(processed_sfe, "nGenes", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top") + custom_theme()
spatial <- Voyager::plotSpatialFeature(processed_sfe, "nGenes", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary",
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) + custom_theme()
violin + spatial

In [ ]:
scater::plotColData(processed_sfe, x = "nCounts", y = "nGenes", colour_by = "in_tissue") + custom_theme()

In [ ]:
mito_ind <- str_detect(rowData(processed_sfe)$symbol, "^mt-")
colData(processed_sfe)$prop_mito <- colSums(counts(processed_sfe)[mito_ind,]) / colData(processed_sfe)$nCounts

In [ ]:
violin <- scater::plotColData(processed_sfe, "prop_mito", x = "in_tissue", colour_by = "in_tissue") +
    theme(legend.position = "top") + custom_theme()
spatial <- Voyager::plotSpatialFeature(processed_sfe, "prop_mito", colGeometryName = "spotPoly",
                              annotGeometryName = "tissueBoundary",
                              image = "lowres", maxcell = 5e4,
                              annot_fixed = list(fill = NA, color = "black")) + custom_theme()
violin + spatial

# Only use in_tissue spots

In [ ]:
sfe_tissue <- processed_sfe[, colData(processed_sfe)$in_tissue]
sfe_tissue <- sfe_tissue[rowSums(counts(sfe_tissue)) > 0,]

In [ ]:
rowData(sfe_tissue)$means <- rowMeans(counts(sfe_tissue))
rowData(sfe_tissue)$vars <- rowVars(counts(sfe_tissue))
# Coefficient of variance
rowData(sfe_tissue)$cv2 <- rowData(sfe_tissue)$vars/rowData(sfe_tissue)$means^2

In [ ]:
scater::plotRowData(sfe_tissue, x = "means", y = "vars", bins = 50) +
    ggplot2::geom_abline(slope = 1, intercept = 0, color = "red") +
    ggplot2::scale_x_log10() + ggplot2::scale_y_log10() +
    ggplot2::scale_fill_distiller(palette = "Blues", direction = 1) +
    ggplot2::annotation_logticks() +
    ggplot2::coord_equal() + custom_theme()

In [ ]:
sfe_tissue <- scuttle::logNormCounts(sfe_tissue)

In [ ]:
dec <- scran::modelGeneVar(sfe_tissue, lowess = FALSE)
hvgs <- scran::getTopHVGs(dec, n = 2000)

In [ ]:
sfe_tissue <- BiocSingular::runPCA(sfe_tissue, ncomponents = 30, subset_row = hvgs, scale = TRUE)

In [ ]:
Voyager::ElbowPlot(sfe_tissue, ndims = 30) + custom_theme()

In [ ]:
Voyager::plotDimLoadings(sfe_tissue, dims = 1:5, swap_rownames = "symbol", ncol = 3) + custom_theme()

In [ ]:
set.seed(29)
colData(sfe_tissue)$cluster <- bluster::clusterRows(
    reducedDim(sfe_tissue, "PCA")[,1:3], 
    BLUSPARAM = SNNGraphParam(cluster.fun = "leiden", 
    cluster.args = list(resolution_parameter = 0.5, objective_function = "modularity"))
)

In [ ]:
scater::plotPCA(sfe_tissue, ncomponents = 5, colour_by = "cluster") + custom_theme()

In [ ]:
Voyager::plotSpatialFeature(sfe_tissue, features = "cluster", colGeometryName = "spotPoly", image_id = "lowres") + custom_theme()

In [ ]:
Voyager::spatialReducedDim(sfe_tissue, "PCA", ncomponents = 5, 
                  colGeometryName = "spotPoly", divergent = TRUE, 
                  diverge_center = 0, ncol = 2, 
                  image_id = "lowres", maxcell = 5e4) + custom_theme()

In [ ]:
set.seed(29)
sfe_tissue <- runUMAP(sfe_tissue, dimred = "PCA", n_dimred = 3)

In [ ]:
plotUMAP(sfe_tissue, colour_by = "cluster") + custom_theme()

In [ ]:
markers <- scran::findMarkers(sfe_tissue, groups = colData(sfe_tissue)$cluster,
                       test.type = "wilcox", pval.type = "all", direction = "up")

In [ ]:
genes_use <- vapply(markers, function(x) rownames(x)[1], FUN.VALUE = character(1))
scater::plotExpression(sfe_tissue, rowData(sfe_tissue)[genes_use, "symbol"], x = "cluster",
               colour_by = "cluster", swap_rownames = "symbol") + custom_theme()

In [ ]:
Voyager::plotSpatialFeature(sfe_tissue, genes_use, colGeometryName = "spotPoly", ncol = 2,
                   swap_rownames = "symbol", image_id = "lowres", maxcell = 5e4) + custom_theme()

In [ ]:
sp <- spotPoly(sfe_tissue)

In [ ]:
dimGeometry(sfe_tissue, "spotPoly", MARGIN = 2) <- sp

In [ ]:
(tb <- annotGeometry(sfe_tissue, "tissueBoundary"))

In [ ]:
plot(st_geometry(tb))
plot(sp, col = "gray", add = TRUE)

In [ ]:
plotSpatialFeature(sfe_tissue, features = "nCounts", 
                   colGeometryName = "spotPoly",
                   annotGeometryName = "tissueBoundary", 
                   aes_use = "color", linewidth = 0.5, fill = NA) + custom_theme()

# Spatial Statistics

The intuition behind spatial statistics is that near things are more closely related than distant things, for example, the weather in Brisbane and the Sunshine Coast versus the weather in Melbourne.

Univariate, bivariate and multivariate spatial correlation measures the degree of spatial dependence or clustering for a single variable, two variables or all variables across different locations. It quantifies whether values of a variable at nearby locations are more similar or dissimilar than expected by chance.

For example, Moran’s I is similar to the Pearson correlation between the value at each location and the average value at its neighbors (but not identical. Just like Pearson correlation, Moran’s I is generally bound between -1 and 1, where positive value indicates positive spatial autocorrelation and negative value indicates negative spatial autocorrelation.

To determine if the spatial autocorrelation is statistically significant, the moran.test() function in spdep can be used. It provides a p-value, but the p-value may not be accurate if the data is not normally distributed. As gene expression data is generally not normally distributed and data normalization doesn’t always work well, we use permutation testing to test the significance of Moran’s I and Geary’s C, wrapping moran.mc() in spdep. The “mc” stands for Monte Carlo. The nsim argument specifies the number of simulations.

Types of spatial correlation:
* Univariate
  * Global
      * Moran's I
      * Geary's C
      * Carrelogram
      * Varigram
  * Local
      * Moran Scatter Plot
      * Local Moran's I
      * Local spatial heteroscedasticity
      * Getis-Ord Gi
* Bivariate
    *  Lee's L
    *  Cross Variogram
* Multivariate
    * MULTISPATI PCA
    * Multivariate local Geary's C

In [ ]:
Voyager::listSFEMethods(variate = "uni", scope = "global")

The "moran.test" and "geary.test" refer to autocorrelation functions that provide a p-value. However, the p-value may not be accurate if the data is not normally distributed, which is often the case with gene expression data. 

The "moran.mc" and "geary.mc" perform permutation testing using a Monte Carlo simulation to calculate the p-value. Permutation testing is a robust approach for assessing the significance of spatial correlation, especially when the data is not normally distributed. It provides a reliable way to determine if the observed spatial patterns are likely to have arisen by chance or if they reflect meaningful spatial relationships.

By randomly shuffling the values of the variable across the locations multiple times (e.g., 999 times) and recalculating Moran's I for each permutation. This creates a reference distribution of Moran's I values under the null hypothesis of no spatial correlation.
The observed Moran's I value is then compared to this reference distribution. If the observed value falls in the extreme tails of the distribution (e.g., top 5% or bottom 5%), it suggests that the spatial correlation is statistically significant and unlikely to have occurred by chance.

If the p-value is less than the chosen significance level (e.g., 0.05), it suggests that the observed spatial correlation is statistically significant. If the p-value is greater than the significance level, the spatial correlation is not considered statistically significant.

In [ ]:
(g <- findSpatialNeighbors(sfe_tissue, MARGIN = 2, method = "tri2nb"))

In [ ]:
spatialGraph(sfe_tissue, "graph1", MARGIN = 2) <- g

In [ ]:
colGraph(sfe_tissue, "visium") <- findVisiumGraph(sfe_tissue, zero.policy = TRUE)

In [ ]:
plot(colGraph(sfe_tissue, "graph1"), coords = spatialCoords(sfe_tissue))

In [ ]:
plot(colGraph(sfe_tissue, "visium"), coords = spatialCoords(sfe_tissue))

In [ ]:
Voyager::calculateMoransI(t(colData(sfe_tissue)[,c("nCounts", "nGenes")]), listw = colGraph(sfe_tissue, "graph1"))

In [ ]:
sfe_tissue <- Voyager::runMoransI(sfe_tissue, features = hvgs, colGraphName = "graph1")

In [ ]:
df <- rowData(sfe_tissue)[hvgs,]

In [ ]:
ord <- order(df$I_sample01, decreasing = TRUE)
df[ord, c("symbol", "I_sample01")]

In [ ]:
df[1:6,1:3]

In [ ]:
(Voyager::plotSpatialFeature(sfe_tissue, rownames(df)[1:6], colGeometryName = "spotPoly", image = "lowres", maxcell = 5e4, 
                             swap_rownames = "symbol", ncol = 2)) + custom_theme()